# 🔥 ♦️ ♥️ 🌟 World Famous 3-player Quantum Poker ♦️ ♥️ 🌟🔥  

### By Xin Kai Lee, Jacob Knight, & Catherine Ji 

## Packages

In [1]:
import gates as gt
import circuit as ct
import random

import matplotlib
# matplotlib.use("TkAgg")
# from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
# from matplotlib.figure import Figure
# import only system from os 
from os import system, name 

# import sleep to show output for some time period 
from time import sleep 
from qiskit.visualization import plot_histogram
from IPython.display import clear_output

## Frontend functions

Main functions for the frontend are in this section.

In [9]:
class player:
    def __init__(self,hand,points):
        self.hand = hand
        self.points = points

def options(c,card,end_sw):
    gate = None
    qubits = [0,1,2]

    if card == "Hadamard":
        print(f"choose target: {qubits}")
        target = int(input())
        gate = gt.HadamardGate(target)
        qubits.remove(target)
    elif card == "XGate":
        print(f"choose target: {qubits}")
        target = int(input())
        gate = gt.XGate(target)
        qubits.remove(target)
    elif card == "CNOT":
        print(f"choose control: {qubits}")
        control = int(input())
        qubits.remove(control)
        print(f"choose target: {qubits}")
        target = int(input())
        gate = gt.CNOTGate(control,target)
        qubits.remove(target)
    elif card == "Toffoli":
        print(f"choose control 1: {qubits}")
        control1 = int(input())
        qubits.remove(control1)
        print(f"choose control 2: {qubits}")
        control2 = int(input())
        qubits.remove(control2)
        print(f"choose target: {qubits}")
        target = int(input())
        gate = gt.ToffoliGate([control1,control2],target)
    elif card == "SWAP":
        print(f"choose control 1: {qubits}")
        control1 = int(input())
        qubits.remove(control1)
        print(f"choose control 2: {qubits}")
        control2 = int(input())
        qubits.remove(control2)
        gate = gt.SWAPGate([control1,control2])
    elif card == "Measure":
        end_sw = end_game_seq(c,end_sw)
        
    return gate, end_sw
    
def play_card(c,player,gate,card):
    c.apply_gate(gate)
    if player.hand[card] == 1:
        player.hand.pop(card)
    else:
        player.hand[card] -= 1

def display_card(player):
    print(player.hand)

def end_game_seq(c,end_sw):
    clear_output(wait=False)
    c.apply_gate(gt.MeasureGate())
    fig = c.draw()
    print(c.draw())
    print("Waiting for results from IonQ . . . . .")
    end_sw = 1
#     result = c.simulate()
    result = c.run()
    print(f"RESULTS: {result.get_counts()}")
    return end_sw

def init_shuffle():
    arr = ["SWAP","Hadamard","Toffoli","XGate","XGate","XGate","CNOT","CNOT","CNOT"]
    random.shuffle(arr)

    h = [{},{},{}]
    for i in range(3):
        curr_p = h[i]
        curr_p["Measure"] = 1
        for j in range(3):
            temp = arr[(3 * i) + j]
            if temp not in curr_p:
                curr_p[arr[(3 * i) + j]] = 1
            else:
                curr_p[arr[(3 * i) + j]] += 1
    
    return h

def ind_round(player_arr,curr_player,control,target):
    
    p1,p2,p3 = player_arr
    if (len(p1.hand) == 0 and len(p2.hand) == 0 and len(p3.hand) == 0):
        end_game_seq()
  
    player = player_arr[curr_player]
    
    while len(player.hand) == 0:
        curr_player += 1
        curr_player = curr_player % 3
        player = player_arr[curr_player]

    display_card(player)
    
#     if player clicks on card:
    card = random.sample(list(player.hand), 1)[0] #change this
    gate_appl = options(card,control,target)
    play_card(player,gate_appl,card)

    display_card(player)
    curr_player += 1
    curr_player = player % 3
    
    return curr_player

def init_func():
    house = 0
    end_sw = 0
    rn = 0
    
    curr_player = 0
    hands = init_shuffle()
    p1 = player(hands[0],0)
    p2 = player(hands[1],0)
    p3 = player(hands[2],0)

    player_arr = [p1,p2,p3]

    c = ct.Circuit()
    return c,house,end_sw,curr_player,player_arr,rn
#initializing

def check_card(card,rn,arr):
    sw = 0
    if (card == "Measure" and rn < 3):
        sw = 1
    if card not in arr:
        sw = 1
        
    return sw

## Main game function

Feel free to play.... if you dare 😈 

In [ ]:
def main():
    print("-----Begin Game-----")
    c,house,end_sw,curr_player,player_arr,rn = init_func()
    
    #Rounds
    while end_sw == 0:
        p1,p2,p3 = player_arr
        if (len(p1.hand) == 0 and len(p2.hand) == 0 and len(p3.hand) == 0):
            end_game_seq()

        player = player_arr[curr_player]

        print(f"*** Player {curr_player} ***")
        print("Press Enter to continue")
        input()

        print("These are your cards")
        cards = player_arr[curr_player].hand
        print(cards)
        fig = c.draw()
        print(c.draw())
        
        sw = 0
        print(f"choose from one option: {player.hand.keys()}")
        choice = input()
        sw = check_card(choice,rn,player.hand)
        
        while sw == 1:
            print("Bad selection, pick another card")
            print(f"choose from one option: {player.hand.keys()}")
            choice = input()
            sw = check_card(choice,rn,player.hand)
        
        gate_appl,end_sw = options(c,choice,end_sw)
        play_card(c,player,gate_appl,choice)
    
        print(f"===END OF TURN, Round: {rn}===")
        
        if end_sw == 0:
            clear_output(wait=False)

        curr_player += 1
        curr_player = curr_player % 3
        rn += 1

main()   

-----Begin Game-----
*** Player 0 ***
Press Enter to continue
